### Importamos las librerias que nos seran utiles

In [925]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns

#### Importamos los datos

In [926]:
sharks = pd.read_csv('/home/julian/Cursos/Ironhack/Proyecto1/shark-data-cleaning/attacks.csv', encoding="ISO-8859-1")

In [927]:
sdf = pd.DataFrame(sharks)

#### Como podemos observar, la mayoria de los datos son no numericos.

In [928]:
sdf.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

### LIMPIEZA HIPOTESIS I

#### Vamos a eliminar las columnas que no son relevantes para el analisis de la primer hipotesis

In [929]:
def keep_columns(x, y):
    relevant = x
    for i in y.columns:
        if i not in relevant:
            y.drop(columns=[i], inplace=True)

In [930]:
keep_columns(['Case Number', 'Type', 'Case Number.1', 'Case Number.2'], sdf)

#### Las columnas 'Case Number', 'Case Number.1' y 'Case Number.2' son relevantes para el analisis, sin embargo, vamos a chequear en que medida los valores de las columnas se parecen para prescindir de alguna de las tres ##DEFINIR FUNCION

In [931]:
"""
def compare(): 
        compare1 = np.where(sdf['Case Number'] == sdf['Case Number.1'], True, False)
        sdf['equal1'] = compare1
        compare2 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)
        sdf['equal2'] = compare2
        compare3 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)
        sdf['equal3'] = compare3
        return print(sdf['equal1'].value_counts()); print(sdf['equal2'].value_counts()); print(sdf['equal3'].value_counts())
"""
            

"\ndef compare(): \n        compare1 = np.where(sdf['Case Number'] == sdf['Case Number.1'], True, False)\n        sdf['equal1'] = compare1\n        compare2 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)\n        sdf['equal2'] = compare2\n        compare3 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)\n        sdf['equal3'] = compare3\n        return print(sdf['equal1'].value_counts()); print(sdf['equal2'].value_counts()); print(sdf['equal3'].value_counts())\n"

In [932]:
compare = np.where(sdf['Case Number.1'] == sdf['Case Number.2'], True, False)

In [933]:
sdf['equal'] = compare

In [934]:
sdf['equal'].value_counts() 

False    19441
True      6282
Name: equal, dtype: int64

In [935]:
sdf.isnull().sum()

Case Number      17021
Type             19425
Case Number.1    19421
Case Number.2    19421
equal                0
dtype: int64

#### SI COMPARAMOS LAS TRES COL ENTRE SI, EN DONDE MENOS DIF DE VAL HAY ES ENTRE CAS NBR Y CAS NBR.2, AUNQUE LA DI ES MINIMA. SIN EMBARGO DE LAS TRES COL LA QUE MENOS CANT DE NULL TIENE ES CASE NBR, NOS QUEDAMOS CON ELLA##DEJAMOS SOLO LAS COLUMNAS CASE NR Y TYPE

In [936]:
keep_columns(['Case Number', 'Type'], sdf)

#### VAMOS A ELIMINAR LOS VALORES QUE NO SEAN FECHA, YA QUE NOS INTERESA VER LA ESTACIONALIDAD. Empezamos por los valores NaN, y luego añadiremos una columna solo con los valores que contengan informacion sobre el mes, ya que nos interesa solo la indfo mensual

In [937]:
sdf.dropna(inplace=True)

In [938]:
def filtracion(x):
    months = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'Jun','07':'Jul','08':'Aug','09':'Sep','10':'Oct','11':'Nov','12':'Dec'}
    m = re.findall('\d{4}.\d{2}.\d{2}', x)
    if m and m[0][5:7] != '00':
        for k, v in months.items():
            if k == m[0][5:7]:
                return v

In [939]:
sdf['Months'] = sdf['Case Number'].apply(filtracion)

#### Borramos los valores nulos para los datos que no tenian registro mensual

In [940]:
sdf.dropna(inplace=True)

#### exportamos el data frame a un csv para realizar las visualizaciones

In [941]:
sdf.to_csv('sdf',index=False)

#### aHORA CONSTRUIMOS UN DATAFRAME CUYAS COLUMNAS CONTENDRAN LAS FRECUENCIAS MENSUALES DE ATAQUES Y LOS MESES CORRESPNDIENTES consideramos todos los tipos  de ataque.

In [942]:
ataques_mensuales = pd.DataFrame(sdf['Months'].value_counts()).reset_index()

#### RENOMBRAMOS LAS COLUMNAS PARA EMPROLIJAR

In [943]:
c = {'Months': 'Monthly Attacks', 'index':'Month'}
ataques_mensuales = ataques_mensuales.rename(columns=c)

In [944]:
ataques_mensuales = ataques_mensuales.reindex([3, 11, 8, 5, 10, 4, 0, 1, 2, 7, 9, 6])

In [945]:
display(ataques_mensuales)

,Month,Monthly Attacks
3,Jan,520
11,Feb,382
8,Mar,409
5,Apr,457
10,May,391
4,Jun,499
0,Jul,668
1,Aug,599
2,Sep,552
7,Oct,441


#### Exportamos el data frame de ataques mensuales para utilizarlo en las visualizaciones

In [946]:
ataques_mensuales.to_csv('ataques_mensuales',index=False)

### LIMPIEZA HIPOTESIS II

In [947]:
sdf2= pd.DataFrame(sharks)

### vamos a quedarnos solo con las columnas que nos interesan para analizar la H2, 'Species ' , 'Injury' y 'Fatal'

In [948]:
keep_columns(['Injury', 'Fatal (Y/N)', 'Species '], sdf2)

### vamos a limpiar las col 'Injury' y 'Fatal'

#### Si bien ambas columnas poseen datos sobre la letalidad, la columna 'Sepecies' es la que mas datos nulos posee. Comenzaremos limpiando esa columna

In [949]:
sdf2['Fatal (Y/N)'].isna().value_counts()

True     19960
False     5763
Name: Fatal (Y/N), dtype: int64

In [950]:
sdf2['Injury'].isna().value_counts()

True     19449
False     6274
Name: Injury, dtype: int64

In [951]:
sdf2['Species '].isna().value_counts()

True     22259
False     3464
Name: Species , dtype: int64

In [952]:
sdf2.dropna(inplace=True)

In [953]:
def filtracion_especies(x):
    m = re.findall('\w+. shark', x)
    if m:
        return (m[0].lower())

In [954]:
sdf2['Identified Specie'] = sdf2['Species '].apply(filtracion_especies)

### Vamos a armar dos data frames, uno con las muertes y otro que considere todos los ataques

### Ataques x especie

In [955]:
ataques = pd.DataFrame(sdf2['Identified Specie'].value_counts()).reset_index()

In [956]:
d = {'Identified Specie': 'Ataques', 'index':'Especie'}

In [957]:
ataques = ataques.rename(columns=d)

#### Como hay muchas categorias de tiburones con tan solo un registro, vamos a considerar solo las que tengan al menos 10 registros de ataque.

In [958]:
ataques = ataques[ataques['Ataques'] > 5]

In [959]:
ataques.shape

(45, 2)

In [960]:
ataques.to_csv('Ataques-especies',index=False)

### Letalidad por especie

#### Como la columna Fatal posee mas valores no nulos que Injury, potencialmente podemos obtener mas info sobre letalidad, vamos a prescindir de Injury. Vamos a quedarnos con los valores en donde hay un 'Y' en Fatal, en la col letalidad

In [961]:
sdf2.drop(columns=['Injury'], inplace=True)

In [962]:
letal = pd.DataFrame(sdf2[sdf2['Fatal (Y/N)'] == 'Y'])

In [963]:
letal2 = pd.DataFrame(letal['Identified Specie'].value_counts()).reset_index()

In [964]:
d = {'Identified Specie': 'Cantidad de Muertes', 'index':'Especie'}
letal2 = letal2.rename(columns=d)

#### Como podemos ver, a partir de la septima fila, los registros son muy, bajos, vamos a recortar el data frame para quedarnos con las especies que mas registros tienen

In [965]:
letal2.drop(range(8, 41), axis=0, inplace=True)

In [966]:
letal2.to_csv('especies-letalidad',index=False)

## HIPOTESIS III 

In [967]:
sdf3 = pd.DataFrame(sharks)

In [968]:
keep_columns(['Country', 'Species '], sdf3)

In [969]:
sdf3['Country'].isna().value_counts()

True     19471
False     6252
Name: Country, dtype: int64

In [970]:
sdf3['Species '].isna().value_counts()

True     22259
False     3464
Name: Species , dtype: int64

#### Como vimos anteriormente, la columna 'Sepecies' es la que mas datos nulos posee. Comenzaremos limpiando esa columna

In [971]:
sdf3.dropna(inplace=True)

In [972]:
def filtracion_especies(x):
    m = re.findall('\w+. shark', x)
    if m:
        return (m[0].lower())

In [973]:
sdf3['Especie'] = sdf3['Species '].apply(filtracion_especies)

In [974]:
sdf3.head()

,Country,Species,Especie
0,USA,White shark,white shark
3,AUSTRALIA,2 m shark,None
4,MEXICO,"Tiger shark, 3m",tiger shark
6,BRAZIL,Tiger shark,tiger shark
7,USA,"Lemon shark, 3'",lemon shark


#### Nos deshacemos de los valores 'None' y la columna 'Species'

In [990]:
keep_columns(['Especie', 'Country'], sdf3)

In [991]:
sdf3.dropna(inplace=True)

In [992]:
display(sdf3)

,Country,Especie
0,USA,white shark
4,MEXICO,tiger shark
6,BRAZIL,tiger shark
7,USA,lemon shark
8,USA,bull shark
...,...,...
6266,SAUDI ARABIA,tipped shark
6267,SAUDI ARABIA,6' shark
6274,ITALY,white shark
6276,BELIZE,tiger shark


In [1018]:
sdf3.groupby('Especie').agg('Country')

In [1013]:
ag = pd.DataFrame(agrupado)

In [1014]:
display(ag)

,0,1
0,TONGA,Country Especie 1678 TONGA small ...
1,AMERICAN SAMOA,Country Especie 4131 AMERIC...
2,ANDAMAN / NICOBAR ISLANDAS,Country Especie 4...
3,ANGOLA,Country Especie 2513 ANGOLA white ...
4,ARGENTINA,Country Especie 4349 ARGENTINA ...
...,...,...
113,USA,Country Especie 0 USA ...
114,VANUATU,Country Especie 1575 VANUATU ...
115,VENEZUELA,Country Especie 1702 VENEZU...
116,VIETNAM,Country Especie 1026 VIETNAM 30k...
